In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_ollama import OllamaEmbeddings
from langchain.vectorstores import FAISS

embedding = OllamaEmbeddings(model="bge-m3")

db = FAISS.load_local("FAISS_SAJU/", embeddings=embedding, allow_dangerous_deserialization=True)

In [3]:
retriever = db.as_retriever(search_kwargs={"k":10})

In [4]:
retriever.invoke("안녕?")

[Document(id='eb472063-3afc-46de-b07c-a8edcb6140db', metadata={'source': 'data/The-Four-Pillars-of-Destiny-Understanding-Character-Relationships-and-Potential-through-Chinese-Astrology.md'}, page_content='|--------------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|'),
 Document(id='9d4e0288-2b18-44c4-99dc-db2e7559be02', metadata={'source': 'data/The-Four-Pillars-of-Destiny-Understanding-Character-Relationships-and-Potential-through-Chinese-Astrology.md'}, page_

### Chain 구축


In [5]:
#from langchain import hub
from langchain_core.prompts import load_prompt

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

from operator import itemgetter

# prompt = hub.pull("teddynote/rag-prompt-chat-history")
prompt = load_prompt("../prompt/rag-prompt-chat-history.yaml")

model = ChatOpenAI(model="gpt-4o-mini")

pdf_chain = {
    "question": itemgetter("question"),
    "context": itemgetter("context"),
    "chat_history": itemgetter("chat_history"),
} | prompt | model | StrOutputParser()


In [6]:
prompt.pretty_print()

You are an AI assistant specializing in Question-Answering (QA) tasks within a Retrieval-Augmented Generation (RAG) system. 
Your primary mission is to answer questions based on provided context or chat history.
Ensure your response is concise and directly addresses the question without any additional narration.

###

You may consider the previous conversation history to answer the question.

# Here's the previous conversation history:
{chat_history}

###

Your final answer should be written concisely (but include important numerical values, technical terms, jargon, and names), followed by the source of the information.

# Steps

1. Carefully read and understand the context provided.
2. Identify the key information related to the question within the context.
3. Formulate a concise answer based on the relevant information.
4. Ensure your final answer directly addresses the question.
5. List the source of the answer in bullet points, which must be a file name (with a page number) or URL 

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# 세션 기록을 저장할 딕셔너리
store = {}

# 세션 ID를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    print(f"[대화 세션ID]: {session_ids}")
    if session_ids not in store:  # 세션 ID가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환

# 대화를 기록하는 RAG 체인 생성
rag_with_history = RunnableWithMessageHistory(
    pdf_chain,
    get_session_history,  # 세션 기록을 가져오는 함수
    input_messages_key="question",  # 사용자의 질문이 템플릿 변수에 들어갈 key
    history_messages_key="chat_history",  # 기록 메시지의 키
)

In [ ]:
question = "How does the balance between the Heavenly Stems and Earthly Branches in my Ba Zi chart affect my personality and life path? 한국어로 답해주세요"
rag_with_history.invoke({"question": question, "context":retriever.invoke(question)}, config={"configurable": {"session_id": "rag123"}},)


[대화 세션ID]: rag123


'당신의 만세도(Ba Zi) 차트에서 천간(Heavenly Stems)과 지지(Earthly Branches) 간의 균형은 성격과 인생 경로에 큰 영향을 미칩니다. 균형이 맞지 않으면, 예를 들어 모든 요소가 음(陰)으로만 구성되어 있거나 특정 요소만 존재하면 사고가 충돌하고 삶이 힘들어질 수 있습니다. 지지는 천을 실현하는 것으로, 각 지지가 하나 또는 여러 개의 숨겨진 천간을 포함할 수 있어 이들이 개인의 심리가 형성되는 데 기여합니다. 따라서 천간과 지지 간의 관계를 이해하는 것이 중요합니다.\n\n**Source**\n- data/The-Four-Pillars-of-Destiny-Understanding-Character-Relationships-and-Potential-through-Chinese-Astrology.md (Page 내용을 종합하여 작성하였습니다.)'

In [ ]:
question = "나의 이전 질문이 뭐였지?"
rag_with_history.invoke({"question": question, "context":retriever.invoke(question)}, config={"configurable": {"session_id": "rag123"}},)

[대화 세션ID]: rag123


'당신의 이전 질문은 "How does the balance between the Heavenly Stems and Earthly Branches in my Ba Zi chart affect my personality and life path?"입니다.\n\n**Source**\n- (없음)'

### 문서관련성 평가

In [7]:
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate

class GradeDocuments(BaseModel):
    """
    A binary score to determine relevance of the retrieved document.
    """
    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes', or 'no'"
    )

llm = ChatOpenAI(model="gpt-4o-mini")

structured_llm_grader = llm.with_structured_output(GradeDocuments)

system = """You are a grader assessing relevance of a retrieved document to a user question. \n 
    If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""

grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved documents: \n\n{document} \n\n User Question: {question}")
    ]
)

retriever_grader = grade_prompt | structured_llm_grader

In [8]:
# 질문 정의
question = "How does the balance between the Heavenly Stems and Earthly Branches in my Ba Zi chart affect my personality and life path?"

# 문서 검색
docs = retriever.invoke(question)

In [9]:
docs[0].metadata

{'source': 'data/The-Four-Pillars-of-Destiny-Understanding-Character-Relationships-and-Potential-through-Chinese-Astrology.md'}

In [10]:
for i, doc in enumerate(docs, 1):
    result = retriever_grader.invoke({
        "question": question,
        "document": doc.page_content
    })
    print(f"[{i}] {result}")

[1] binary_score='yes'
[2] binary_score='yes'
[3] binary_score='yes'
[4] binary_score='yes'
[5] binary_score='yes'
[6] binary_score='no'
[7] binary_score='yes'
[8] binary_score='yes'
[9] binary_score='no'
[10] binary_score='no'


### 답변 생성 체인

In [11]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import load_prompt

# LangChain Hub에서 RAG 프롬프트를 가져와 사용
# prompt = hub.pull("teddynote/rag-prompt")
prompt = load_prompt("../prompt/rag_chain.yaml")
llm = ChatOpenAI(model="gpt-4o-mini")

def format_docs(docs):
    return "\n\n".join(
        [
            #f'<document><content>{doc.page_content}</content><source>{doc.metadata["source"]}</source><page>{doc.metadata["page"]+1}</page></document>'
            f'<document><content>{doc.page_content}</content><source>{doc.metadata["source"]}</source></document>'
            for doc in docs
        ]
    )
    
rag_chain = prompt | llm | StrOutputParser()

In [12]:
# CRAG에서는 개별평가를 한다.
question = "How does the balance between the Heavenly Stems and Earthly Branches in my Ba Zi chart affect my personality and life path? 한국어로 답해주세요"
generation = rag_chain.invoke({"context": format_docs(retriever.invoke(question)), "question": question})
print(generation)

당신의 Ba Zi 차트에서 천간(Heavenly Stems)과 지지(Earthly Branches) 간의 균형은 성격과 인생 경로에 중요한 영향을 미칩니다. 천간은 타고난 특성과 성격을 나타내며, 지지는 세상과의 상호작용을 반영합니다. 불균형한 천간만으로 성격이 혼란스러울 수 있고, 이는 생의 어려움을 초래할 수 있습니다. 예를 들어, 모든 요소가 음(阴)인 경우, 사고방식에서 갈등이 생길 수 있습니다. 따라서 두 요소의 조화는 당신의 정체성과 인생의 방향을 결정짓는 중요한 역할을 합니다.

**Source**
- data/The-Four-Pillars-of-Destiny-Understanding-Character-Relationships-and-Potential-through-Chinese-Astrology.md


### 쿼리 재작성(Question Re-writer)

In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

# Query Rewrite 시스템 프롬프트
system = """
You are a question rewriter specialized in Four Pillars of Destiny (사주팔자, Saju) analysis. 
Your job is to convert a user's vague or informal question into a well-structured and contextually clear query suitable for knowledge retrieval or reasoning based on Saju principles.

You must:
- Clarify ambiguous expressions (e.g. "운이 좋을까?" → "Based on the user's Saju, is there good fortune in this year?")
- Include necessary details if missing, such as "birth year, month, day, and time" if implied or inferable
- Ensure the rewritten question is answerable using Saju logic (e.g. 천간/지지, 오행, 대운, 세운, 용신 등)
- Do **not** invent personal details. Only elaborate if the user's intent is logically implied.
- 

Input: a natural language question(Korean-language)
Output: a precise, Saju-context-aware question that elicits a Korean-language response

"""

re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human",
         "Here is the initial question: \n {question} \n\n Formulate an improved question."),
    ]
)

question_rewriter = re_write_prompt | llm | StrOutputParser()

In [14]:
question = "How does the balance between the Heavenly Stems and Earthly Branches in my Ba Zi chart affect my personality and life path? 한국어로 답해주세요"
question_rewriter.invoke({"question": question})

'내 사주팔자에서 천간과 지지의 균형이 내 성격과 삶의 경로에 어떤 영향을 미치는지 분석해 주실 수 있나요?'

### 웹 검색 도구

In [15]:
from langchain.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults()

In [16]:
question = "2024년 노벨문학상 수상자는?"
results = web_search_tool.invoke({"query": question})
print(results)

[{'title': '노벨 문학상 수상자 목록 - 위키백과, 우리 모두의 백과사전', 'url': 'https://ko.wikipedia.org/wiki/%EB%85%B8%EB%B2%A8_%EB%AC%B8%ED%95%99%EC%83%81_%EC%88%98%EC%83%81%EC%9E%90_%EB%AA%A9%EB%A1%9D', 'content': '2024년 현재, 노벨 문학상은 121명의 개인에게 수여되었다.[5] 18명의 여성이 노벨 문학상을 수상했는데, 이는 노벨 평화상 다음으로 많은 수치이다.[6][7] 2024년 기준으로 영어권 노벨 문학상 수상자는 29명이며, 그 뒤를 이어 프랑스어권 16명, 독일어권 14명의 수상자가 있다. 프랑스는 가장 많은 노벨 문학상 수상자를 배출한 국가이다.\n연대별 수상자\n[편집]\n1900년대\n[편집]\n| 연도 | 수상자 | 나라 | 언어 | 장르 | 대표작과 수상 이유 |\n| --- | --- | --- | --- | --- | --- |\n| 1901 |  | 쉴리 프뤼돔 |  프랑스 | 프랑스어 | 시 | 구절과 시\n"고상한 이상주의의 증거를 주는, 그의 낭만적인 기질과 예술적인 완전함 그리고 감성과 지성의 보기 드문 결합을 특별히 인정하여 이 상을 드립니다."[8] |\n| 1902 |  | 테오도어 몸젠 |  독일 제국 | 독일어 | 역사 | 로마의 역사 [...] 1901년부터 2024년까지 121명의 노벨 문학상 수상자들의 출신 국가는 다음과 같다.\n| 국가 | 수상자 수 |\n| --- | --- |\n|  프랑스 | 16 |\n|  미국 | 13 |\n|  영국 | 13 |\n|  독일 | 9 |\n|  스웨덴 | 8 |\n|  스페인 | 6 |\n|  이탈리아 | 6 |\n|  폴란드 | 6 |\n|  러시아\n 소련 | 5 |\n|  노르웨이 | 4 |\n|  아일랜드 | 4 |\n|  덴마크 | 3 |\n|  그리스 | 2 |\n|  남아프리카공화국 | 2 |\n|  스위스 | 2 |\n| 

# 상태 정의

In [17]:
from typing import Annotated, TypedDict

class GraphState(TypedDict):
    question: Annotated[str, "The question to answer"]
    generation: Annotated[str, "The generation from the LLM"]
    web_search: Annotated[str, "Whether to add search"]
    documents: Annotated[list[str], "The documents retrieved"]

# 노드

In [18]:
# 문서 검색 노드
def retrieve(state: GraphState):
    print("\n=== Retrieve ===\n")
    
    question = state["question"]
    documents = retriever.invoke(question)
    return {"documents": documents}

In [19]:
# 답변 생성 노드
def generate(state: GraphState):
    print("\n=== Generation ===\n")

    question = state["question"]
    documents = state["documents"]

    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"generation": generation}

In [20]:
# 문서 평가 노드
def grade_documents(state: GraphState):
    print("\n=== Check Document Relevance to Question ===\n")
    
    question = state["question"]
    documents = state["documents"]

    filtered_docs = []
    relevance_doc_count = 0

    for d in documents:
        score = retriever_grader.invoke(
            {"question": question, "document": d.page_content} # documents가 아니다. documents에서 내용만 뽑아낸 docyment르 받은것이다. retriever_grader prompt에도 그렇게 설정했다.
        )
        grade = score.binary_score

        if grade == "yes":
            print("\n=== Grade: Document Relevant ===\n")
            
            filtered_docs.append(d)
            relevance_doc_count += 1
        else:
            print("\n=== Grade: Document Not Relevant ===\n")
            continue

    # 관련 문서가 없으면 웹검색 수행: relevance_doc_count == 0이면 web_search가 Yes고 그게 아니라면 no로 반환.
    web_search = "Yes" if relevance_doc_count == 0 else "No"
    return {"documents": filtered_docs, "web_search": web_search}

In [21]:
# 쿼리 재작성
def query_rewrite(state: GraphState):
    print(question)
    print("\n=== Query Rewrite ===\n")

    question = state["question"]
    
    better_question = question_rewriter.invoke({"question": question})
    return {"question": better_question}

In [22]:
web_search_tool.invoke({"query": question})

[{'title': '노벨 문학상 수상자 목록 - 위키백과, 우리 모두의 백과사전',
  'url': 'https://ko.wikipedia.org/wiki/%EB%85%B8%EB%B2%A8_%EB%AC%B8%ED%95%99%EC%83%81_%EC%88%98%EC%83%81%EC%9E%90_%EB%AA%A9%EB%A1%9D',
  'content': '2024년 현재, 노벨 문학상은 121명의 개인에게 수여되었다.[5] 18명의 여성이 노벨 문학상을 수상했는데, 이는 노벨 평화상 다음으로 많은 수치이다.[6][7] 2024년 기준으로 영어권 노벨 문학상 수상자는 29명이며, 그 뒤를 이어 프랑스어권 16명, 독일어권 14명의 수상자가 있다. 프랑스는 가장 많은 노벨 문학상 수상자를 배출한 국가이다.\n연대별 수상자\n[편집]\n1900년대\n[편집]\n| 연도 | 수상자 | 나라 | 언어 | 장르 | 대표작과 수상 이유 |\n| --- | --- | --- | --- | --- | --- |\n| 1901 |  | 쉴리 프뤼돔 |  프랑스 | 프랑스어 | 시 | 구절과 시\n"고상한 이상주의의 증거를 주는, 그의 낭만적인 기질과 예술적인 완전함 그리고 감성과 지성의 보기 드문 결합을 특별히 인정하여 이 상을 드립니다."[8] |\n| 1902 |  | 테오도어 몸젠 |  독일 제국 | 독일어 | 역사 | 로마의 역사 [...] 1901년부터 2024년까지 121명의 노벨 문학상 수상자들의 출신 국가는 다음과 같다.\n| 국가 | 수상자 수 |\n| --- | --- |\n|  프랑스 | 16 |\n|  미국 | 13 |\n|  영국 | 13 |\n|  독일 | 9 |\n|  스웨덴 | 8 |\n|  스페인 | 6 |\n|  이탈리아 | 6 |\n|  폴란드 | 6 |\n|  러시아\n 소련 | 5 |\n|  노르웨이 | 4 |\n|  아일랜드 | 4 |\n|  덴마크 | 3 |\n|  그리스 | 2 |\n|  남아프리카공화국 | 2 |\n|  스위스 | 2 |

In [23]:
from langchain_core.documents import Document
# 웹 검색 노드
def web_search(state: GraphState):
    print("\n=== Web Search ===\n")

    question = state["question"]
    documents = state["documents"]

    docs = web_search_tool.invoke({"query": question})
    
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)
    documents.append(web_results)

    return {"documents": documents}

# 조건부 엣지 활용

In [24]:
def decide_to_generate(state: GraphState):
    print("\n=== Assess graded documents ===\n")

    web_search = state["web_search"]

    if web_search == "Yes":
        print(
            "===[Decision: All Document Are Not Relevant to Question, Query Rewrite]==="
        )
        return "web_search_node"
    else:
        print("===[Decision: Generate]===")
        return "generate"

In [40]:
from langgraph.graph import START, END, StateGraph
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

workflow = StateGraph(GraphState)

# 노드
workflow.add_node("retrieve", retrieve)
workflow.add_node("grade_documents",grade_documents)
workflow.add_node("generate", generate)
workflow.add_node("query_rewrite",query_rewrite)
workflow.add_node("web_search_node", web_search)

# 엣지 연결
# workflow.add_edge(START, "retrieve")
# workflow.add_edge("retrieve", "grade_documents")
# workflow.add_conditional_edges(
#     "grade_documents",
#     decide_to_generate,
#     {
#         "query_rewrite": "query_rewrite",
#         "generate": "generate"
#     }
# )

workflow.add_edge(START, "query_rewrite")
workflow.add_edge("query_rewrite", "retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_edge("grade_documents", "web_search_node")
workflow.add_edge("web_search_node", "generate")
workflow.add_edge("generate", END)

app = workflow.compile(checkpointer=memory)

In [41]:
from langchain_teddynote.graphs import visualize_graph

visualize_graph(app)

[ERROR] Visualize Graph Error: Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 502.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`


In [42]:
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import stream_graph, invoke_graph, random_uuid

config = RunnableConfig(recursion_limit=20, configurable={"thread_id": random_uuid()})

inputs = {
    "question": "삼성전자가 개발한 생성형 AI의 이름은?"
}

stream_graph(app, inputs, config)


=== Query Rewrite ===


🔄 Node: query_rewrite 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
삼성전자가 개발한 생성형 AI의 이름은 무엇인지에 대한 정보가 필요합니다.
=== Retrieve ===


=== Check Document Relevance to Question ===


🔄 Node: grade_documents 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
{"binary_score":"no"}
=== Grade: Document Not Relevant ===

{"binary_score":"no"}
=== Grade: Document Not Relevant ===

{"binary_score":"no"}
=== Grade: Document Not Relevant ===

{"binary_score":"no
=== Grade: Document Not Relevant ===

"}{"binary_score":"no"}
=== Grade: Document Not Relevant ===

{"binary_score":"yes"}
=== Grade: Document Relevant ===

{"binary_score":"no"}
=== Grade: Document Not Relevant ===

{"binary_score":"no"}
=== Grade: Document Not Relevant ===

{"binary_score":"no"}
=== Grade: Document Not Relevant ===

{"binary_score":"no"}
=== Grade: Document Not Relevant ===


=== Web Search ===


=== Generation ===


🔄 Node: generate 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
삼성전자가

In [47]:
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import stream_graph, invoke_graph, random_uuid

config = RunnableConfig(recursion_limit=20, configurable={"thread_id": 1})
question = "How does the balance between the Heavenly Stems and Earthly Branches in my Ba Zi chart affect my personality and life path? 1995년 3월 28일 오시에 태어났어"
inputs = {
    "question": question
}

stream_graph(app, inputs, config)


=== Query Rewrite ===


🔄 Node: query_rewrite 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
내 사주팔자의 천간과 지지의 균형이 나의 성격과 인생 경로에 어떤 영향을 미치는지 분석해 주세요. (출생일: 1995년 3월 28일, 출생시간: 오전 5시)
=== Retrieve ===


=== Check Document Relevance to Question ===


🔄 Node: grade_documents 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
{"binary_score":"yes"}
=== Grade: Document Relevant ===

{"binary_score":"no"}
=== Grade: Document Not Relevant ===

{"binary_score":"no"}
=== Grade: Document Not Relevant ===

{"binary_score":"yes
=== Grade: Document Relevant ===

"}{"binary_score":"no"}
=== Grade: Document Not Relevant ===

{"binary_score":"no
=== Grade: Document Not Relevant ===

"}{"binary_score":"no"}
=== Grade: Document Not Relevant ===

{"binary_score":"no"}
=== Grade: Document Not Relevant ===

{"binary_score":"no"}
=== Grade: Document Not Relevant ===

{"binary_score":"no
=== Grade: Document Not Relevant ===

"}
=== Web Search ===


=== Generation ===


🔄 Node: generate 🔄
- - - - - - -